In [53]:

infobox_label_soup = infobox_label_soups_list[0]
[f'infobox_label_soup.{fn}' for fn in dir(infobox_label_soup) if 'next' in fn.lower()]

['infobox_label_soup.fetchNextSiblings', 'infobox_label_soup.findAllNext', 'infobox_label_soup.findNext', 'infobox_label_soup.findNextSibling', 'infobox_label_soup.findNextSiblings', 'infobox_label_soup.find_all_next', 'infobox_label_soup.find_next', 'infobox_label_soup.find_next_sibling', 'infobox_label_soup.find_next_siblings', 'infobox_label_soup.next', 'infobox_label_soup.nextGenerator', 'infobox_label_soup.nextSibling', 'infobox_label_soup.nextSiblingGenerator', 'infobox_label_soup.next_element', 'infobox_label_soup.next_elements', 'infobox_label_soup.next_sibling', 'infobox_label_soup.next_siblings']

In [ ]:

mask_series = (catholic_papacy_df.age_at_inauguration > 100)
df = catholic_papacy_df[mask_series]
display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

papal_title


In [195]:

catholic_papacy_df.loc['Pope Adrian II', 'year_reign_begin'] = 867
catholic_papacy_df.loc['Pope Adrian II', 'age_at_inauguration'] = (
    catholic_papacy_df.loc['Pope Adrian II', 'year_reign_begin'] - catholic_papacy_df.loc['Pope Adrian II', 'year_of_birth']
)

In [ ]:

    for row_index, row_series in catholic_papacy_df.iterrows():
        monarch_name = row_series.monarch_name
        wikipedia_url = row_series.wikipedia_url
        try:
            page_obj = wikipedia.page(title=monarch_name)
            page_html = page_obj.html()
            page_soup = bs(page_html, 'html.parser')
        except:
            page_soup = nu.get_page_soup(wikipedia_url, verbose=False)
        
        # Get the birth year
        try:
            for th_soup in page_soup.find_all(
                name='th',
                attrs={'scope': 'row', 'class': 'infobox-label'},
                recursive=True,
                text='Born'
            ):
                td_soup = th_soup.find_next_sibling(name='td', attrs={'class': 'infobox-data'})
                year_of_birth = get_first_year_element(' '.join(list(td_soup.stripped_strings)))
                catholic_papacy_df.loc[row_index, 'year_of_birth'] = year_of_birth
        except:
            tables_df_list = nu.get_page_tables(wikipedia_url, verbose=False)
            df = tables_df_list[0].copy()
            mask_series = (df.iloc[:, 0] == 'Born')
            year_of_birth = get_first_year_element(df[mask_series].iloc[:, 1].squeeze())
            catholic_papacy_df.loc[row_index, 'year_of_birth'] = year_of_birth
        
        # Get the death year
        year_of_death = np.nan
        try:
            for th_soup in page_soup.find_all(
                name='th',
                attrs={'scope': 'row', 'class': 'infobox-label'},
                recursive=True,
                text='Died'
            ):
                td_soup = th_soup.find_next_sibling(name='td', attrs={'class': 'infobox-data'})
                year_of_death = get_first_year_element(' '.join(list(td_soup.stripped_strings)))
                catholic_papacy_df.loc[row_index, 'year_of_death'] = year_of_death
        except:
            tables_df_list = nu.get_page_tables(wikipedia_url, verbose=False)
            df = tables_df_list[0].copy()
            mask_series = (df.iloc[:, 0] == 'Born')
            year_of_death = get_first_year_element(df[mask_series].iloc[:, 1].squeeze())
            catholic_papacy_df.loc[row_index, 'year_of_death'] = year_of_death
        
    # Fix missing birth years
    mask_series = catholic_papacy_df.year_of_birth.isnull()
    df = catholic_papacy_df[mask_series]
    if df.shape[0]:
        display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)
        tables_url = 'https://en.wikipedia.org/wiki/List_of_English_monarchs'
        tables_df_list = nu.get_wiki_tables(tables_url, verbose=False)
        for monarch_name in df.monarch_name:
            for i, tables_df in enumerate(tables_df_list):
                df1 = tables_df.applymap(lambda x: monarch_name in str(x), na_action='ignore')
                srs = df1.sum()
                columns_list = srs[srs!=0].index.tolist()
                if columns_list:
                    for cn in columns_list:
                        mask_series = (df1[cn] == True)
                        for idx in df1[mask_series].index:
                            year_of_birth = get_first_year_element(tables_df.iloc[idx]['Birth'])
                            print(monarch_name, year_of_birth)
                            mask_series = (catholic_papacy_df.monarch_name == monarch_name)
                            catholic_papacy_df.loc[mask_series, 'year_of_birth'] = year_of_birth
    mask_series = catholic_papacy_df.year_of_birth.isnull()
    assert not catholic_papacy_df[mask_series].shape[0], 'You still have empty year of birth values'
    catholic_papacy_df.year_of_birth = catholic_papacy_df.year_of_birth.map(lambda x: int(x))
    
    catholic_papacy_df['office_rank'] = catholic_papacy_df.index.map(lambda x: x+1)
    catholic_papacy_df = catholic_papacy_df.rename(columns={'year_reign_begin': 'year_reign_begin', 'year_term_end': 'year_reign_end'})
    
    # Get the infobox data
    rows_list = []
    for row_index, row_series in catholic_papacy_df.iterrows():
        monarch_name = row_series.monarch_name
        wikipedia_url = row_series.wikipedia_url
        try:
            page_obj = wikipedia.page(title=monarch_name)
            page_html = page_obj.html()
            page_soup = bs(page_html, 'html.parser')
        except:
            page_soup = nu.get_page_soup(wikipedia_url, verbose=False)
        row_dict = row_series.to_dict()
        for th_soup in page_soup.find_all(
            name='th',
            attrs={'scope': 'row', 'class': 'infobox-label'},
            recursive=True,
        ):
            key = th_soup.text
            td_soup = th_soup.find_next_sibling(name='td', attrs={'class': 'infobox-data'})
            value = td_soup.text
            
            value = re.sub(r'[ \u2009\xa0]+', ' ', str(value).strip())
            key = re.sub(r'[ \u2009\xa0]+', '_', key.strip())
            if key.startswith('Issue'): key = 'Issue'
            elif key.startswith('Predece'): key = 'Predecessors'
            elif key.startswith('Succe'): key = 'Successors'
            elif key.startswith('Spouse'): key = 'Spouses'
            elif key.startswith('Regen'): key = 'Regency'
            elif key.startswith('Reign'): key = 'Reigns'
            elif key.startswith('Alma'): key = 'Alma_Mater'
            elif key.startswith('Repealed'): key = 'Repealed_By'
            elif key.startswith('Years'): key = 'Years_of_Active_Service'
            elif key.startswith('Commands'): key = 'Commands_Held'
            elif key.startswith('Service'): key = 'Service_Branch'
            row_dict[key] = value
        rows_list.append(row_dict)

    infobox_df = pd.DataFrame(rows_list)
    mask_series = infobox_df.year_reign_begin.isnull()
    infobox_df.loc[mask_series, 'year_reign_begin'] = infobox_df[mask_series].Reigns.map(lambda x: get_first_year_element(str(x).split('–')[0]))
    mask_series = infobox_df.year_reign_end.isnull()
    infobox_df.loc[mask_series, 'year_reign_end'] = infobox_df[mask_series].Reigns.map(lambda x: get_first_year_element(str(x).split('–')[-1]))
    for cn in ['office_rank']+[cn for cn in infobox_df.columns if cn.startswith('year_')]:
        infobox_df[cn] = pd.to_numeric(infobox_df[cn], errors='coerce')
    
    # Get the generation name and turnings archetype
    for row_index, row_series in infobox_df.iterrows():
        year_of_birth = row_series.year_of_birth
        mask_series = (generations_df.birth_year_begin <= year_of_birth) & (generations_df.birth_year_end >= year_of_birth)
        df = generations_df[mask_series]
        if df.shape[0]:
            generation_name = df.index.tolist()[-1]
            infobox_df.loc[row_index, 'generation_name'] = generation_name
            turnings_archetype = df.turnings_archetype.tolist()[-1]
            infobox_df.loc[row_index, 'turnings_archetype'] = turnings_archetype

    # Get the turning info
    for row_index, row_series in infobox_df.iterrows():
        year_reign_begin = row_series.year_reign_begin

        # Get turning number
        mask_series = (turnings_df.turning_year_begin <= year_reign_begin) & (turnings_df.turning_year_end >= year_reign_begin)
        df = turnings_df[mask_series]
        if df.shape[0]:
            turning_number = df.turning_number.tolist()[-1]
            infobox_df.loc[row_index, 'turning_number'] = turning_number

            # Get turning type
            mask_series = (turning_numbers_df.index == turning_number)
            df = turning_numbers_df[mask_series]
            if df.shape[0]:
                turning_type = df.turning_type.tolist()[-1]
                infobox_df.loc[row_index, 'turning_type'] = turning_type

        # Get turning name
        mask_series = (turnings_df.turning_year_begin <= year_reign_begin) & (turnings_df.turning_year_end >= year_reign_begin)
        df = turnings_df[mask_series]
        if df.shape[0]:
            turning_name = df.index.tolist()[-1]
            infobox_df.loc[row_index, 'turning_name'] = turning_name
    
    # Fix various indexes and columns
    # infobox_df.turning_number = infobox_df.turning_number.astype(int)
    infobox_df['age_at_inauguration'] = (
        infobox_df.year_reign_begin - infobox_df.year_of_birth
    )
    
    # Fix bad inauguration ages
    mask_series = infobox_df.age_at_inauguration.isin([-9, 419, 617])
    df = infobox_df[mask_series]
    if df.shape[0]:
        
        # Fix Edward IV of England
        url = 'https://en.wikipedia.org/wiki/Edward_IV_of_England'
        infobox_df.loc[39, 'wikipedia_url'] = url
        infobox_df.loc[40, 'wikipedia_url'] = url
        mask_series = (infobox_df.monarch_name == 'Edward IV of England')
        df = infobox_df[mask_series]

        # Get the infobox data
        rows_list = []
        for row_index, row_series in df.iterrows():
            monarch_name = row_series.monarch_name
            wikipedia_url = row_series.wikipedia_url
            page_soup = nu.get_page_soup(wikipedia_url, verbose=False)

            # Get the keys and set the values
            for th_soup in page_soup.find_all(
                name='th',
                attrs={'scope': 'row', 'class': 'infobox-label'},
                recursive=True,
            ):
                key = th_soup.text
                td_soup = th_soup.find_next_sibling(name='td', attrs={'class': 'infobox-data'})
                value = td_soup.text

                value = re.sub(r'[ \u2009\xa0]+', ' ', str(value).strip())
                key = re.sub(r'[ \u2009\xa0]+', '_', key.strip())
                if key.startswith('Issue'): key = 'Issue'
                elif key.startswith('Predece'): key = 'Predecessors'
                elif key.startswith('Succe'): key = 'Successors'
                elif key.startswith('Spouse'): key = 'Spouses'
                elif key.startswith('Regen'): key = 'Regency'
                elif key.startswith('Reign'): key = 'Reigns'
                elif key.startswith('Alma'): key = 'Alma_Mater'
                elif key.startswith('Repealed'): key = 'Repealed_By'
                elif key.startswith('Years'): key = 'Years_of_Active_Service'
                elif key.startswith('Commands'): key = 'Commands_Held'
                elif key.startswith('Service'): key = 'Service_Branch'
                infobox_df.loc[row_index, key] = value
            infobox_df.loc[row_index, 'year_of_birth'] = 1442

        infobox_df.loc[39, 'year_reign_begin'] = 1461
        infobox_df.loc[39, 'year_reign_end'] = 1470
        infobox_df.loc[39, 'age_at_inauguration'] = (
            infobox_df.loc[39, 'year_reign_begin'] - infobox_df.loc[39, 'year_of_birth']
        )

        infobox_df.loc[40, 'year_reign_begin'] = 1471
        infobox_df.loc[40, 'year_reign_end'] = 1483
        infobox_df.loc[40, 'age_at_inauguration'] = (
            infobox_df.loc[40, 'year_reign_begin'] - infobox_df.loc[40, 'year_of_birth']
        )
    
        # Fix Elizabeth II
        mask_series = (infobox_df.monarch_name == 'Elizabeth II')
        df = infobox_df[mask_series]

        # Get the infobox data
        rows_list = []
        for row_index, row_series in df.iterrows():
            monarch_name = row_series.monarch_name
            wikipedia_url = row_series.wikipedia_url
            page_soup = nu.get_page_soup(wikipedia_url, verbose=False)

            # Get the keys and set the values
            for th_soup in page_soup.find_all(
                name='th',
                attrs={'scope': 'row', 'class': 'infobox-label'},
                recursive=True,
            ):
                key = th_soup.text
                td_soup = th_soup.find_next_sibling(name='td', attrs={'class': 'infobox-data'})
                value = td_soup.text

                value = re.sub(r'[ \u2009\xa0]+', ' ', str(value).strip())
                key = re.sub(r'[ \u2009\xa0]+', '_', key.strip())
                if key.startswith('Issue'): key = 'Issue'
                elif key.startswith('Predece'): key = 'Predecessors'
                elif key.startswith('Succe'): key = 'Successors'
                elif key.startswith('Spouse'): key = 'Spouses'
                elif key.startswith('Regen'): key = 'Regency'
                elif key.startswith('Reign'): key = 'Reigns'
                elif key.startswith('Alma'): key = 'Alma_Mater'
                elif key.startswith('Repealed'): key = 'Repealed_By'
                elif key.startswith('Years'): key = 'Years_of_Active_Service'
                elif key.startswith('Commands'): key = 'Commands_Held'
                elif key.startswith('Service'): key = 'Service_Branch'
                infobox_df.loc[row_index, key] = value
            infobox_df.loc[row_index, 'year_of_birth'] = 1926
            infobox_df.loc[row_index, 'year_of_death'] = 2022

        infobox_df.loc[69, 'year_reign_begin'] = 1952
        infobox_df.loc[69, 'year_reign_end'] = 2022
        infobox_df.loc[69, 'age_at_inauguration'] = (
            infobox_df.loc[69, 'year_reign_begin'] - infobox_df.loc[69, 'year_of_birth']
        )
        
        # Fix Edward VII
        mask_series = (infobox_df.monarch_name == 'Edward VII')
        df = infobox_df[mask_series]

        # Get the infobox data
        rows_list = []
        for row_index, row_series in df.iterrows():
            monarch_name = row_series.monarch_name
            wikipedia_url = row_series.wikipedia_url
            page_soup = nu.get_page_soup(wikipedia_url, verbose=False)

            # Get the keys and set the values
            for th_soup in page_soup.find_all(
                name='th',
                attrs={'scope': 'row', 'class': 'infobox-label'},
                recursive=True,
            ):
                key = th_soup.text
                td_soup = th_soup.find_next_sibling(name='td', attrs={'class': 'infobox-data'})
                value = td_soup.text

                value = re.sub(r'[ \u2009\xa0]+', ' ', str(value).strip())
                key = re.sub(r'[ \u2009\xa0]+', '_', key.strip())
                if key.startswith('Issue'): key = 'Issue'
                elif key.startswith('Predece'): key = 'Predecessors'
                elif key.startswith('Succe'): key = 'Successors'
                elif key.startswith('Spouse'): key = 'Spouses'
                elif key.startswith('Regen'): key = 'Regency'
                elif key.startswith('Reign'): key = 'Reigns'
                elif key.startswith('Alma'): key = 'Alma_Mater'
                elif key.startswith('Repealed'): key = 'Repealed_By'
                elif key.startswith('Years'): key = 'Years_of_Active_Service'
                elif key.startswith('Commands'): key = 'Commands_Held'
                elif key.startswith('Service'): key = 'Service_Branch'
                infobox_df.loc[row_index, key] = value
            infobox_df.loc[row_index, 'year_of_birth'] = 1841
            infobox_df.loc[row_index, 'year_of_death'] = 1910

        infobox_df.loc[65, 'year_reign_begin'] = 1901
        infobox_df.loc[65, 'year_reign_end'] = 1910
        infobox_df.loc[65, 'age_at_inauguration'] = (
            infobox_df.loc[65, 'year_reign_begin'] - infobox_df.loc[65, 'year_of_birth']
        )
    
    infobox_df = infobox_df.set_index('monarch_name')
    if ('index' in infobox_df.columns): infobox_df = infobox_df.drop('index', axis='columns')

In [84]:

[cn for cn in english_and_british_monarchs_df.columns if 'reign' in cn.lower()]

['year_reign_begin', 'year_reign_end', 'Reigns']

In [10]:

items_dict = check_4_doubles(infobox_df.columns.tolist()).sort_values('max_similarity', ascending=False).head(10).to_dict()
print(items_dict['first_item'])
print(items_dict['second_item'])

{21: 'Junior_king', 0: 'Reigns', 26: 'Co-monarch', 27: 'Acclamation', 6: 'Spouses', 9: 'Father', 11: 'Coronation', 29: 'Preceded_by', 1: 'Predecessors', 35: 'Occupation'}
{21: 'Senior_king', 0: 'Religion', 26: 'Monarch', 27: 'Occupation', 6: 'House', 9: 'Mother', 11: 'Citation', 29: 'Amended_by', 1: 'Successors', 35: 'Education'}


In [ ]:

# Convert the DataFrame to a list of row dictionaries
rows_list = []
for dictionary in df.to_dict('records'):
    for key, value in dictionary.items():
        row_dict = {}
        value = re.sub(r'[ \u2009\xa0]+', ' ', str(value).strip())
        key = re.sub(r'[ \u2009\xa0]+', '_', key.strip())
        if key.startswith('Issue'): key = 'Issue'
        elif key.startswith('Predece'): key = 'Predecessors'
        elif key.startswith('Succe'): key = 'Successors'
        elif key.startswith('Spouse'): key = 'Spouses'
        elif key.startswith('Regen'): key = 'Regency'
        elif key.startswith('Reign'): key = 'Reigns'
        elif key.startswith('Alma'): key = 'Alma_Mater'
        elif key.startswith('Repealed'): key = 'Repealed_By'
        elif key.startswith('Years'): key = 'Years_of_Active_Service'
        elif key.startswith('Commands'): key = 'Commands_Held'
        elif key.startswith('Service'): key = 'Service_Branch'
        row_dict[key] = value
        rows_list.append(row_dict)
df = pd.DataFrame(rows_list)

In [ ]:

fn = lambda x: 'nan' == x
srs = infobox_df.applymap(fn, na_action='ignore').sum()
columns_list = srs[srs>0].index.tolist()
if columns_list:
    df = infobox_df[columns_list].sample(min(4, infobox_df.shape[0])).dropna(axis='columns', how='all').T
    display(df)
    print(df.to_dict())